# NOTEBOOK FOR ANALYSING THE `SYNTHEA` DATA

- GOAL IS TO FIND STRONG REASONS BEHIND PATIENTS READMISSION

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys, os
sys.path.append(os.path.abspath(r"C:\Users\Rano's PC\Machine\github_repo_cloned\my-personal-projects\synthea readmission analysis"))

import src.plots as P
import src.stat_tests as S

plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')

In [22]:
data = pd.read_csv(r"C:\Users\Rano's PC\Machine\github_repo_cloned\my-personal-projects\synthea readmission analysis\data\processed_data\processed_synthea_data.csv")

In [23]:
data.head()

,encounter_start,encounterclass,description,base_encounter_cost,payer_coverage,reasondescription,duration,total_procedures,avg_procedure_cost,total_medicines,...,age_as_2022,marital,race,ethnicity,patient_gender,provider_gender,utilization,is_readmission,condition_duration,careplan_duration
0,2010-01-23 17:45:28,AMBULATORY,Encounter for symptom,129.16,54.16,Acute bronchitis (disorder),1_month,0.0,0.00,0.0,...,38,MARRIED,WHITE,NONHISPANIC,M,F,3199,0,NaN,730 days
1,2012-01-23 17:45:28,WELLNESS,General examination of patient (procedure),129.16,129.16,unknown,1_month,0.0,0.00,0.0,...,38,MARRIED,WHITE,NONHISPANIC,M,F,33,0,NaN,NaN
2,2001-05-01 15:02:18,AMBULATORY,Encounter for symptom,129.16,0.00,Sinusitis (disorder),1_month,0.0,0.00,0.0,...,32,MARRIED,WHITE,HISPANIC,M,M,2585,0,NaN,NaN
3,2011-07-28 15:02:18,WELLNESS,General examination of patient (procedure),129.16,0.00,unknown,1_month,0.0,0.00,0.0,...,32,MARRIED,WHITE,HISPANIC,M,F,58,0,NaN,NaN
4,2010-07-27 12:58:08,WELLNESS,General examination of patient (procedure),129.16,129.16,unknown,1_month,1.0,726.51,0.0,...,29,MARRIED,WHITE,NONHISPANIC,M,M,59,0,NaN,NaN


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53346 entries, 0 to 53345
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   encounter_start          53346 non-null  object 
 1   encounterclass           53346 non-null  object 
 2   description              53346 non-null  object 
 3   base_encounter_cost      53346 non-null  float64
 4   payer_coverage           53346 non-null  float64
 5   reasondescription        53346 non-null  object 
 6   duration                 53346 non-null  object 
 7   total_procedures         53346 non-null  float64
 8   avg_procedure_cost       53346 non-null  float64
 9   total_medicines          53346 non-null  float64
 10  avg_base_medicine_cost   53346 non-null  float64
 11  avg_total_medicine_cost  53346 non-null  float64
 12  reasons_for_medications  53346 non-null  float64
 13  is_ongoing_condition     53346 non-null  float64
 14  total_conditions      

In [6]:
data.shape

(53346, 29)

In [7]:
data.describe().round(2)

,base_encounter_cost,payer_coverage,total_procedures,avg_procedure_cost,total_medicines,avg_base_medicine_cost,avg_total_medicine_cost,reasons_for_medications,is_ongoing_condition,total_conditions,is_ongoing_careplan,total_careplans,age_as_2022,utilization,is_readmission
count,53346.00,53346.00,53346.00,53346.00,53346.0,53346.00,53346.00,53346.00,53346.00,53346.00,53346.00,53346.00,53346.00,53346.00,53346.00
mean,128.75,62.00,0.66,2129.45,0.8,162.61,1431.68,0.50,-0.80,0.16,-0.91,0.07,55.66,1329.70,0.39
std,4.58,43.61,1.70,5596.18,1.2,406.75,9931.92,0.61,0.53,0.41,0.38,0.25,25.87,1427.73,0.49
min,77.49,0.00,0.00,0.00,0.0,0.00,0.00,0.00,-1.00,0.00,-1.00,0.00,0.00,1.00,0.00
25%,129.16,17.49,0.00,0.00,0.0,0.00,0.00,0.00,-1.00,0.00,-1.00,0.00,35.00,86.00,0.00
50%,129.16,69.16,0.00,0.00,1.0,6.80,7.22,0.00,-1.00,0.00,-1.00,0.00,58.00,821.00,0.00
75%,129.16,89.16,1.00,602.04,1.0,263.49,1783.74,1.00,-1.00,0.00,-1.00,0.00,73.00,2159.00,1.00
max,129.16,129.16,21.00,187968.69,10.0,6901.17,1296594.95,4.00,1.00,7.00,1.00,2.00,112.00,4828.00,1.00


# ***Statistical insights from numerical columns***

- max value of `procedure_cost` is very high

- max value of `average_total_medicine_cost` is too high

- `utilization` column is spreaded from 1 to 4828

- `age` column have values from 0 to 112 as of 2022

In [20]:
data.select_dtypes(include='number').nunique().reset_index(name = 'total_unique_values')

,index,total_unique_values
0,base_encounter_cost,2
1,payer_coverage,15
2,total_procedures,10
3,avg_procedure_cost,15909
4,total_medicines,11
5,avg_base_medicine_cost,11010
6,avg_total_medicine_cost,11986
7,reasons_for_medications,5
8,is_ongoing_condition,3
9,total_conditions,7


In [18]:
data.select_dtypes(include='object').nunique().reset_index(name = 'total_unique_values')

,index,total_unique_values
0,encounter_start,50029
1,encounterclass,6
2,description,50
3,reasondescription,81
4,duration,2
5,condition_start,5353
6,condition_end,2923
7,careplan_start,2919
8,careplan_end,1634
9,marital,3


In [ ]:
small_cat_cols = ['encounterclass', 'duration', 'marital','race', 'ethnicity', 'patient_gender', 
                  'provider_gender','base_encounter_cost','reasons_for_medications','is_ongoing_condition','is_ongoing_careplan','total_careplans']

In [ ]:
date_cols = ['encounter_start','condition_start']